In [1]:
# set environment
!pip install torch==1.7 torchtext==0.8

     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 7.0MB 29.5MB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext import data
from torchtext import datasets

import random
import time

torch.__version__, torchtext.__version__

('1.7.0', '0.8.0')

## Preparing Data

In [3]:
%%time

# set random seed for reproducibility
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# downlaod and split dataset (trian, val, test)
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)
train_val_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_val_data.split(random_state = random.seed(SEED))

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
aclImdb_v1.tar.gz:   0%|          | 81.9k/84.1M [00:00<01:42, 816kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 36.6MB/s]
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


CPU times: user 1min 44s, sys: 10.7 s, total: 1min 55s
Wall time: 1min 58s


In [4]:
# check the type and size of dataset
print(f'>>> type : {type(train_data)}')
print(f'>>> Number of training examples: {len(train_data)}')   # 17500 (35%)
print(f'>>> Number of validation examples: {len(valid_data)}') # 7500  (15%)
print(f'>>> Number of testing examples: {len(test_data)}')     # 25000 (50%)
print()

# check one sample data
tmp_ex = train_val_data.examples[0]
tmp_dict = vars(tmp_ex)
print('< example data >')
print('>>> type :', type(tmp_ex))
for key in tmp_dict:
  print(f'>>> {key} : {tmp_dict[key]}')

>>> type : <class 'torchtext.data.dataset.Dataset'>
>>> Number of training examples: 17500
>>> Number of validation examples: 7500
>>> Number of testing examples: 25000

< example data >
>>> type : <class 'torchtext.data.example.Example'>
>>> text : ['This', 'is', 'one', 'of', 'those', 'movies', 'that', ',', 'after', 'watching', 'it', ',', 'you', 'will', 'keep', 'thinking', 'about', 'and', 'coming', 'back', 'to', 'even', 'months', 'after', 'viewing', '.', 'The', 'acting', 'is', 'spectacular', 'for', 'starring', 'two', 'children', '(', 'I', 'usually', 'hate', 'movies', 'with', 'whiny', 'kid', 'acting', ')', '.', 'Bill', 'Paxton', 'is', 'awesome', ',', 'his', 'directorial', 'debut', 'is', 'even', 'better', 'than', 'what', 'I', 'expected', 'of', 'an', 'actor', 'whose', 'most', 'memorable', 'line', 'is', '"', 'Game', 'over', ',', 'man', '!', 'Game', 'over', '!', '"', '(', 'from', 'the', 'movie', '"', 'Aliens").<br', '/><br', '/>The', 'best', 'part', 'about', 'the', 'movie', 'is', 'the', 'd

In [5]:
%%time

# build vocabulary
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f">>> Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f">>> Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(f">>> Top 20 common tokens :{TEXT.vocab.freqs.most_common(20)}")
print()
print('<itos and stoi>')
print('>>> itos :', TEXT.vocab.itos[:10])
print('>>> stoi :', LABEL.vocab.stoi)

>>> Unique tokens in TEXT vocabulary: 25002
>>> Unique tokens in LABEL vocabulary: 2
>>> Top 20 common tokens :[('the', 201483), (',', 191071), ('.', 165466), ('and', 108901), ('a', 108890), ('of', 99815), ('to', 92913), ('is', 76451), ('in', 61037), ('I', 54352), ('it', 53477), ('that', 49157), ('"', 43935), ("'s", 43037), ('this', 42289), ('-', 36343), ('/><br', 35252), ('was', 34941), ('as', 30317), ('movie', 29866)]

<itos and stoi>
>>> itos : ['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
>>> stoi : defaultdict(None, {'neg': 0, 'pos': 1})
CPU times: user 1.6 s, sys: 17 ms, total: 1.62 s
Wall time: 1.62 s


In [6]:
# create the iterators
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)
device, train_iterator, len(train_iterator), len(train_data)/BATCH_SIZE

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


(device(type='cpu'),
 274,
 273.4375)

## Build the Model

In [7]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

        
    def forward(self, text):
        # text = [sent len, batch size]
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        
        # output = [sent len, batch size, hid dim]
        # hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))


INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100  # 50-250
HIDDEN_DIM = 256     # 100-500
OUTPUT_DIM = 1       # No of labels

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'>>> The model has {num_parameters:,} trainable parameters')

>>> The model has 2,592,105 trainable parameters


## Train the Model

In [8]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [9]:
%%time

# generate optimizer & loss function
optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss().to(device)

# train model
N_EPOCHS = 5
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Epoch Time: 15m 38s
	Train Loss: 0.694 | Train Acc: 50.28%
	 Val. Loss: 0.695 |  Val. Acc: 50.10%
Epoch: 02 | Epoch Time: 15m 34s
	Train Loss: 0.693 | Train Acc: 49.93%
	 Val. Loss: 0.696 |  Val. Acc: 50.18%
Epoch: 03 | Epoch Time: 15m 37s
	Train Loss: 0.693 | Train Acc: 49.90%
	 Val. Loss: 0.696 |  Val. Acc: 50.84%
Epoch: 04 | Epoch Time: 15m 35s
	Train Loss: 0.693 | Train Acc: 49.84%
	 Val. Loss: 0.695 |  Val. Acc: 49.91%
Epoch: 05 | Epoch Time: 15m 37s
	Train Loss: 0.693 | Train Acc: 50.29%
	 Val. Loss: 0.696 |  Val. Acc: 50.72%
CPU times: user 1h 17min 44s, sys: 12.5 s, total: 1h 17min 56s
Wall time: 1h 18min 3s


## Load and Test saved model

In [10]:
%%time

model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss: 0.710 | Test Acc: 47.19%
CPU times: user 36.2 s, sys: 409 ms, total: 36.6 s
Wall time: 36.6 s
